In [ ]:
from deep_orderbook.replayer import Replayer
from deep_orderbook.shapper import BookShapper
import numpy as np
from pylab import rcParams

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import clear_output
rcParams['figure.figsize'] = 20, 4

In [ ]:
shapper = await BookShapper.create()
file_replayer = Replayer('../crypto-trading/data/L2')
replay = file_replayer.replayL2('ETHBTC', shapper)
b, a, tp, tr = next(replay)

In [ ]:
%%time
for i in range(20):
    b, a, tp, tr = next(replay)
print(f"bids:\n{b.head()}")
print(f"asks:\n{a.head()}")
print(f"prices:\n{tp}")
print(f"trades:\n{tr}")

In [ ]:
shapper = await BookShapper.create()
file_replayer = Replayer('../crypto-trading/data/L2')
areplay = file_replayer.replayL2_async('ETHBTC', shapper)
b, a, tp, tr = await areplay.__anext__()

In [ ]:
for i in range(30):
    b, a, tp, tr = await areplay.__anext__()
    b1, a1, tp1, tr1 = next(replay)
    if not b1.head(100).equals(b.head(100)):
        print(i, 'b')
    if not a1.head(100).equals(a.head(100)):
        print(i, 'a')

In [ ]:
b1

In [ ]:
b

In [ ]:
print(f"bids:\n{b.head()}")
print(f"asks:\n{a.head()}")
print(f"prices:\n{tp}")
print(f"trades:\n{tr}")

In [ ]:
replayers = [file_replayer.replayL2('ETHBTC', await BookShapper.create()),
             file_replayer.replayL2('BTCUSDT', await BookShapper.create()),
            ]
multi_replay = file_replayer.multireplayL2(replayers)
d = next(multi_replay)

In [ ]:
for i in range(100):
    d = next(multi_replay)
d

In [ ]:
gen = file_replayer.accumulate_array(markets=['ETHBTC','BTCUSDT'])
_ = next(gen)

In [ ]:
LENGTH = 128
x = []
for n,sec in enumerate(gen):
    allim = []
    for symb, data in sec.items():
        arr = np.stack(data['bs'][-LENGTH:])
        im = arr
        im[:,:,0] /= 10
        im += 0.5
        allim.append(im)
    allim = np.concatenate(allim, axis=1)
    if n % 1 == 0:
        plt.figure(figsize=(15,11))
        toshow = allim.transpose(1,0,2)
        toshow = np.clip(toshow, 0, 1)
        plt.imshow(toshow, origin="lower");
        clear_output(wait=True)
        plt.show()
    pass

In [ ]:
genacc = file_replayer.accumulate_array(markets=['ETHBTC'])
next(genacc)

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
js = [['0.21654', '0.24654'],['0.21654', '0.24654'],['0.21654', '0.24654'],['0.21654', '0.24654'],['0.21654', '0.24654'],['0.21654', '0.24654']]

In [ ]:
%%timeit
pd.DataFrame(js, columns=['price', 'size']).astype(np.float64).set_index('price')

In [ ]:
%%timeit
[[float(e[0]), float(e[1])] for e in js]